In [ ]:
from PIL import Image
from wordcloud import WordCloud
import matplotlib.pyplot as plt
def plot_wordcloud(text, max_words=200, background_image=None, background_color='white', width=800, height=400):
    if background_image:
        mask = np.array(Image.open(background_image))
    wordcloud = WordCloud(width=width, height=height, background_color=background_color, max_words=max_words, collocations=False, mask=mask)
    wordcloud.generate(text)
    plt.figure(figsize=(10,8))
    plt.imshow(wordcloud)
    plt.axis("off")
    plt.show()


plot_wordcloud("".join([str(word) for word in clean_corpus]),background_image="/Users/luciegabagnou/Documents/MOSEF/PYTHON/projet_trip_advisor/sentiment-analysis-tripadvisor/TripAdvisor.jpg")